In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..")))
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from src.data_loader import load_sales, load_purchases, load_products
from src.demand_forecast import calculate_forecast
from src.stock_manager import calculate_min_stock
from src.purchase_suggester import suggest_purchases,generate_purchase_suggestion
from src.supplier_selector import select_suppliers


In [2]:
# === Opciones disponibles ===
coverage_days_options = [7, 15, 21, 30]
forecast_methods = ["mean", "rolling", "weighted", "trend"]

# === Widgets ===
coverage_dropdown = widgets.Dropdown(
    options=coverage_days_options,
    value=7,
    description="Cobertura (días):",
    style={'description_width': 'initial'}
)

method_dropdown = widgets.Dropdown(
    options=forecast_methods,
    value="mean",
    description="Método forecast:",
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description="Generar Sugerido",
    button_style="success"
)

output = widgets.Output()

# === Función a ejecutar ===
def run_forecast(b):
    with output:
        output.clear_output()
        try:
            coverage_days = coverage_dropdown.value
            method = method_dropdown.value

            print(f"⚡ Ejecutando forecast con {coverage_days} días y método '{method}'...\n")

            # === Cargar datos ===
            sales = load_sales("../data/sales.csv")
            stock = load_products("../data/products.csv")
            purchases = load_purchases("../data/compras.csv")  # ojo: revisa si es otro loader

            # === Forecast ===
            forecast = calculate_forecast(
                sales,
                days=60,      # últimos 60 días para suavizar outliers
                method=method,
                window=7
            )

            # === Sugerido ===
            suggestion = generate_purchase_suggestion(
                sales,
                stock,
                coverage_days=coverage_dropdown.value
            )

            # === Proveedores ===
            suppliers = select_suppliers(purchases)

            final = suggestion.merge(suppliers, on="product_id", how="left")

            if final.empty:
                print("⚠️ No hay datos para mostrar (revisa tu CSV o filtros).")
            else:
                display(final.head(20))

        except Exception as e:
            print("❌ Error en la ejecución:", e)

# Conectar botón a función
run_button.on_click(run_forecast)

# Mostrar widgets
display(coverage_dropdown, method_dropdown, run_button, output)

Dropdown(description='Cobertura (días):', options=(7, 15, 21, 30), style=DescriptionStyle(description_width='i…

Dropdown(description='Método forecast:', options=('mean', 'rolling', 'weighted', 'trend'), style=DescriptionSt…

Button(button_style='success', description='Generar Sugerido', style=ButtonStyle())

Output()

,product_id,current_stock,min_stock,suggested_purchase,best_supplier,best_cost
0,0000004494,5.0,9.666667,4.666667,NaN,NaN
1,0000004524,2.0,8.272727,6.272727,NaN,NaN
2,0000004526,4.0,9.333333,5.333333,NaN,NaN
3,0000004532,4.0,10.694444,6.694444,TAITAB S.A.S.,1.250000
4,0000004533,9.0,13.275862,4.275862,VERA LOZA SAMUEL ALEJANDRO,0.726042
5,0000004538,0.0,15.272727,15.272727,NaN,NaN
6,0000004539,13.0,10.500000,0.000000,VERA LOZA SAMUEL ALEJANDRO,0.752500
7,0000004548,0.0,7.636364,7.636364,NaN,NaN
8,0000004583,1.0,7.000000,6.000000,NaN,NaN
9,0000004588,141.0,17.937500,0.000000,NaN,NaN


,product_id,current_stock,min_stock,suggested_purchase,best_supplier,best_cost
0,0000004494,5.0,9.666667,4.666667,NaN,NaN
1,0000004524,2.0,8.272727,6.272727,NaN,NaN
2,0000004526,4.0,9.333333,5.333333,NaN,NaN
3,0000004532,4.0,10.694444,6.694444,TAITAB S.A.S.,1.250000
4,0000004533,9.0,13.275862,4.275862,VERA LOZA SAMUEL ALEJANDRO,0.726042
5,0000004538,0.0,15.272727,15.272727,NaN,NaN
6,0000004539,13.0,10.500000,0.000000,VERA LOZA SAMUEL ALEJANDRO,0.752500
7,0000004548,0.0,7.636364,7.636364,NaN,NaN
8,0000004583,1.0,7.000000,6.000000,NaN,NaN
9,0000004588,141.0,17.937500,0.000000,NaN,NaN


,product_id,current_stock,min_stock,suggested_purchase,best_supplier,best_cost
0,0000004494,5.0,9.666667,4.666667,NaN,NaN
1,0000004524,2.0,8.272727,6.272727,NaN,NaN
2,0000004526,4.0,9.333333,5.333333,NaN,NaN
3,0000004532,4.0,10.694444,6.694444,TAITAB S.A.S.,1.250000
4,0000004533,9.0,13.275862,4.275862,VERA LOZA SAMUEL ALEJANDRO,0.726042
5,0000004538,0.0,15.272727,15.272727,NaN,NaN
6,0000004539,13.0,10.500000,0.000000,VERA LOZA SAMUEL ALEJANDRO,0.752500
7,0000004548,0.0,7.636364,7.636364,NaN,NaN
8,0000004583,1.0,7.000000,6.000000,NaN,NaN
9,0000004588,141.0,17.937500,0.000000,NaN,NaN
